In [0]:
!git clone https://github.com/kafitimi/glowing-enigma.git
import os
os.chdir('glowing-enigma')
!pip install -r requirements.txt

In [0]:
from google_drive_downloader import GoogleDriveDownloader as gdd
destination, file_id = '/content/glowing-enigma/somerup.zip', '1ObSBgKDFh6W_o8qBO_X6BmICFx8EZYVa'
gdd.download_file_from_google_drive(file_id,destination,overwrite=True,unzip=False)
!unzip /content/glowing-enigma/somerup 

In [0]:
destination, file_id = '/content/glowing-enigma/somerpds.zip', '1AhMn0pceO6Lg7KkTD_GM-o3uolq4uxWd'
gdd.download_file_from_google_drive(file_id,destination,overwrite=True,unzip=False)
!unzip /content/glowing-enigma/somerpds -d /content/glowing-enigma/rpds


In [0]:
!python get_matrix.py 010301_20-1М--.plx

In [0]:
!python get_rpd.py 010301_20-1М--.plx rpd_example.yaml

In [0]:
#!python get_fos.py 010301_20-1М--.plx fos.docx

In [0]:
# document.paragraphs[183]._element.attrib['{http://schemas.openxmlformats.org/wordprocessingml/2006/main}rsidR']

In [0]:
# ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
# ns_pfx = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
# document._element.findall('.//w:p[@w:rsidR="006B4494"]', ns)

In [0]:
# document.tables[7].tag

In [0]:
#!ls
#os.chdir('/content/glowing-enigma/rpds')


In [0]:
import docx
from docx.enum.text import WD_ALIGN_PARAGRAPH 
import pandas as pd
import numpy as np
df = pd.DataFrame()

header = ['Министерство науки и высшего образования Российской Федерации',
        'Федеральное государственное автономное образовательное учреждение',
        'высшего образования',
        '«СЕВЕРО-ВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ УНИВЕРСИТЕТ ИМЕНИ М.К. АММОСОВА»',
        'Институт математики и информатики', 
        '\n\n\n\n\n\nФонд оценочных средств дисциплины']

middle = ['\nкод дисциплины и название дисциплины', #bold
        '\nдля программы магистратуры/бакалавриата',
        'по направлению подготовки',
        'УГС Название направления подготовки  -- 38.04.05 Бизнес-информатика',
        'Направленность программы: профиль',
        '\n\nФорма контроля:', #bold
        'экзамен/зачет/зачет со оценкой/курсовая работа', #bold
        '\n\nАвторы: Михайлов В.Д., д. филос. наук, профессор кафедры философии СВФУ']
middleBold = [True,False,False,False,False,True,True,False]

footer = ['\n\n\n\n\nЯкутск 2019']

metodmater = ['6.3. Методические материалы, определяющие процедуры оценивания',
'Текущий контроль успеваемости с БРС состоит из:',
'– контрольные срезы, устанавливаемые учебными подразделениями;',
'– рубежный срез, проводимый после окончания зачетной недели, за день до начала',
'экзаменационной сессии, который определяет общую сумму баллов, набранную обучающимся в течение всего семестра по данной дисциплине.',
'Промежуточная аттестация осуществляется в конце семестра после завершения учебных занятий в форме: зачетов, зачетов с оценкой, оценки отчета по практике, экзаменов.',
'Оценка знаний, умений и навыков производится согласно положению о балльно-рейтинговой системе в СВФУ.']



table61 = ['Коды','оцениваемых','Индикаторы','достижения',
 'Показатель','оценивания','(по','п.1.2.РПД)','Шкалы оценивания',
 'сформированности']
table62 = ['Оцениваемый','показатель','(ЗУВ)','Тема','(темы)',
           'Образец','типового','тестового','задания','(вопроса)']
tableBooks = ['Автор','название','издания','издательство','литературы',
              'информационных ресурсов','гриф','НБ СВФУ','библиотека',
              'экземпляров','Электронные','ЭБС','ЭБ СВФУ'] 

def preceding_paragraph(x):
    ''' Определяет тип таблицы и добавляет название параграфа '''
    global table_flazhok 
    if sum([word in x for word in table61])>5:
        fos_doc.add_paragraph('6. Фонд оценочных средств для проведения промежуточной аттестации обучающихся по дисциплине').bold = True
        fos_doc.paragraphs[-1].runs[0].bold = True 
        fos_doc.add_paragraph('6.1. Показатели, критерии и шкала оценивания').bold = True
        fos_doc.paragraphs[-1].runs[0].bold = True 
        table_flazhok = True
        return True
    if sum([word in x for word in table62])>5:
        fos_doc.add_paragraph('6.2. Примерные контрольные задания (вопросы) для промежуточной аттестации').bold = True
        fos_doc.paragraphs[-1].runs[0].bold = True 
        table_flazhok = True
        return True
    if sum([word in x for word in tableBooks])>5:
        table_flazhok = False
    return table_flazhok

def findout_fos(documento):
    ''' Выдергивает все абзацы между литературой и "Примерные контрольные задания" '''
    text_at_start = ['Примерные контрольные задания (вопросы']
    text_at_end = ['учебной литературы', 'iprbook', 'lanbook', 'НБ СВФУ',
                   'информационно-телекоммуникационной сети']
    flag, result_text = False, ''
    for paragraph in documento.paragraphs:
        if any(key_word in paragraph.text for key_word in text_at_end):
          flag = False
        if flag:
          result_text += (paragraph.text+'\n')
        if text_at_start[0] in paragraph.text:
          flag = True
    result_text = result_text.replace('\n\n','\n').replace('  ',' ',1000)    
    return result_text

def table2list(x):
    ''' Читает таблицу из РПД в список'''
    ls =[]
    try:
        for row in x.rows:
            for cell in row.cells:
                celltext = ''
                for paragraph in cell.paragraphs:
                    celltext += '\n' + paragraph.text
                ls.append(celltext)
        return [ls, len(x.rows), len(x.columns)]
    except IndexError:
        return [[' '], 1, 1]

def list2docx(ls, row, column):
    ''' Из списка восстанавливает таблицу, объединяя ячейки со совпадающими текстами'''
    if len(ls)!=row*column:
        return 

    df = pd.DataFrame(np.array(ls).reshape(row,column))  #reshape to the table shape
    word_table = fos_doc.add_table(rows = row, cols = column, style='Table Grid')
    for ind_row in range(0,row,1):
        for ind_col in range(0,column,1):
            cell = word_table.cell(ind_row,ind_col)
            if ind_row>0:
                 cell2 = word_table.cell(ind_row-1,ind_col)
                 if cell2.text == df.iloc[ind_row,ind_col]:
                     cell.text = ''
                     cell.merge(cell2)
                 else:
                     cell.text = df.iloc[ind_row,ind_col]    
            else:
                 cell.text = df.iloc[ind_row,ind_col]  
    fos_doc.add_paragraph(' ')                                      
    return 

fos_doc = docx.Document()
for filename in os.listdir():
    if filename.endswith(".docx"):
        ''' титульная страница '''
        fos_doc.add_page_break() 
        fos_doc.add_paragraph('\n'.join(header))
        fos_doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
        middle[0] = filename[:4]
        middle[3] = '01.03.01 Математика' 
        middle[6] = 'экзамен'
        for ind,line in enumerate(middle):
            fos_doc.add_paragraph(line)
            fos_doc.paragraphs[-1].runs[0].bold=middleBold[ind]
            fos_doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
        fos_doc.add_paragraph('\n'.join(footer)) 
        fos_doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
        fos_doc.add_page_break() 

        ''' флажок, чтобы забрать все таблицы между двумя группами ключевых слов '''
        table_flazhok = False
        rpd_doc = docx.Document(filename)
        for table in rpd_doc.tables:
            tablen = table2list(table)
            if preceding_paragraph(' '.join(tablen[0][:6])):
                list2docx(*tablen)
        ''' таблицы все включены, теперь забираем все абзацы '''
        text_heap = findout_fos(rpd_doc)
        fos_doc.add_paragraph(text_heap) 
        if not 'Методические материалы, определяющие' in text_heap:
             fos_doc.add_paragraph('\n'.join(metodmater)) 


fos_doc.save('fozsh.docx')


In [0]:
os.chdir('/content/glowing-enigma/rpds')
from docx import Document
import re
import numpy as np


 


def findout_fos_ZUV(documento):
    text_in_table = ['знать', 'уметь', 'владеть', 
                     'знает', 'умеет', 'владеет', 
                     '(ЗУВ)', 'п.1.2.']
    tablez = ''
    columns_text = []
    columns_nums = []
    for table in documento.tables:
        flag, tablelist = False, []
        max_col_num = 0
        for row in table.rows:
            rowlist = []
            try:
                for cell in row.cells:
                    celltext = ''
                    for paragraph in cell.paragraphs:
                        celltext += ('\n'+paragraph.text)
                    rowlist.append(celltext)
                if len(rowlist)>max_col_num:
                    max_col_num = len(rowlist)
                if rowlist:               
                    tablelist.append(rowlist)
            except IndexError:
                print('cannot read a table')    
            
        if max_col_num>0:
            tablelist2 = [] 
            for row in tablelist:
                if len(row)<max_col_num:
                    tablelist2.append(row.extend([' ']*(max_col_num-len(row)))) 
                else:
                    if row:
                        tablelist2.append(row)
            if tablelist2[-1] is None:
                 tablelist2=tablelist2[:-1]
            tablelist =  [list(line) for line in zip(*tablelist2)]
            coltext = ['\n'.join(list(dict.fromkeys(col))).replace('\n\n','\n').replace('  ',' ',1000) for col in tablelist]
            columns_text += coltext
            columns_nums += [len(re.findall('(ЗУВ)', col.lower()))*10 + sum([len(re.findall(key_word, col.lower())) for key_word in text_in_table]) for col in coltext]
    maximum = max(columns_nums)
    return [columns_text[i] for i, occurence in enumerate(columns_nums) if occurence > maximum * 0.8]

def marked(wholecolumn):
    marks = ['excellent:','good:','fair:','bad:']
    shift = 2
    for k in range(len(wholecolumn)):
        if 'оценивани' in wholecolumn[k]:
            shift = k + 1

    if len(wholecolumn)<2:
         return '\n'.join(wholecolumn)
    else:
         return '\n'.join([marks[(i-shift) % 4]+celltext for i,celltext in enumerate(wholecolumn)])    


def findout_fos_marks(documento):
    text_in_table = ['шкалы оценивания', 'сформированности', 'компетенций', 
                     'критерии', 'критерий', '(дескрипторы)', 
                     'оценивания', 'критерий оценивания', 
                     'критерии оценивания', 'имеет','не имеет', 'знает','владеет']
    tablez = ''
    columns_text = []
    columns_nums = []
    for table in documento.tables:
        flag, tablelist = False, []
        max_col_num = 0
        for row in table.rows:
            rowlist = []
            try:
                for cell in row.cells:
                    celltext = ''
                    for paragraph in cell.paragraphs:
                        celltext += ('\n'+paragraph.text)
                    rowlist.append(celltext)
                if len(rowlist)>max_col_num:
                    max_col_num = len(rowlist)
                if rowlist:               
                    tablelist.append(rowlist)
            except IndexError:
                print('cannot read a table')    
            
        if max_col_num>0:
            tablelist2 = [] 
            for row in tablelist:
                if len(row)<max_col_num:
                    tablelist2.append(row.extend([' ']*(max_col_num-len(row)))) 
                else:
                    if row:
                        tablelist2.append(row)
            if tablelist2[-1] is None:
                 tablelist2=tablelist2[:-1]
            tablelist =  [list(line) for line in zip(*tablelist2)]
            # coltext = ['\n'.join(list(dict.fromkeys(col))).replace('\n\n','\n').replace('  ',' ',1000) for col in tablelist]
            
            coltext = [list(dict.fromkeys(col)) for col in tablelist]
            #print(coltext)

            coltext = [marked(col) for col in coltext]

            columns_text += coltext
            columns_nums += [sum([len(re.findall(key_word, col.lower())) for key_word in text_in_table]) for col in coltext]
    maximum = max(columns_nums)
    return [columns_text[i] for i, occurence in enumerate(columns_nums) if occurence > maximum * 0.8]


for filename in os.listdir():
    if filename.endswith(".docx"): 
        f = open(filename, 'rb')
        document = Document(f)
        print('=========================================')   
        print(filename)   
        marks_yaml = '-----\n'.join(findout_fos_marks(document)).replace('\n','\n  ')
        marks_yaml = marks_yaml.replace('  excellent:','\nотлично:')
        marks_yaml = marks_yaml.replace('  good:','\nхорошо:')
        marks_yaml = marks_yaml.replace('  fair:','\nудовлетворительно:')
        marks_yaml = marks_yaml.replace('  bad:','\nнеудовлетворительно:')    

        ZUV_yaml = '-----\n'.join(findout_fos_ZUV(document)).replace('\n','\n  ')

        print(ZUV_yaml)   

        got_fos = marks_yaml + '\n' + ZUV_yaml
        f.close()
        f2 = open(filename[:-4]+'txt',"w+") 
        f2.write(got_fos)

        # print(findout_fos_table(document))

os.chdir('/content/glowing-enigma/rpds')
!zip -R foo '*.txt'
os.chdir('/content/glowing-enigma')

In [0]:
dd=[1,2,3,3,4,4]
dd[:5]

[1, 2, 3, 3, 4]

In [0]:
""" Генерация ФОС """
import sys
from typing import Dict

import docx
from docx.table import Table
from docxtpl import DocxTemplate

import difflib
import pandas as pd
import numpy as np

import core
 


def check_args() -> None:
    """ Проверка аргументов командной строки """
    if len(sys.argv) != 3:
        print('Синтаксис:\n\tpython {0} <руп> <фос>'.format(*sys.argv))
        sys.exit()


def fill_table_1(template: DocxTemplate, context: Dict[str, any]) -> None:
    """ Заполнение таблиц с формами контроля """
    control_fancy_name = {
        core.CT_EXAM: 'Экзамен',
        core.CT_CREDIT_GRADE: 'Зачет с оценкой',
        core.CT_CREDIT: 'Зачет',
        core.CT_COURSEWORK: 'Курсовой проект',
    }

    plan: core.EducationPlan = context['plan']
    if plan.degree == core.BACHELOR:
        core.remove_table(template, 1)
    elif plan.degree == core.MASTER:
        core.remove_table(template, 2)
    table: Table = template.get_docx().tables[1]

    row_number = 0
    for competence in sorted(plan.competence_codes.values(), key=core.Competence.repr):
        core.add_table_rows(table, 1)
        row = len(table.rows) - 1
        row_number += 1
        core.set_cell_text(table, row, 0, core.CENTER, str(row_number))
        core.set_cell_text(table, row, 1, core.JUSTIFY, competence.code + ' ' + competence.description)
        table.cell(row, 1).merge(table.cell(row, len(table.columns) - 1))
        subjects = [plan.subject_codes[s] for s in competence.subjects]
        for subject in sorted(subjects, key=core.Subject.repr):
            core.add_table_rows(table, 1)
            row = len(table.rows) - 1
            row_number += 1
            core.set_cell_text(table, row, 0, core.CENTER, str(row_number))
            core.set_cell_text(table, row, 1, core.JUSTIFY, subject.code + ' ' + subject.name)
            for number, semester in subject.semesters.items():
                controls = [control_fancy_name[c] for c in semester.control]
                core.set_cell_text(table, row, number + 1, core.CENTER, ', '.join(controls))

    core.fix_table_borders(table)


def table2list(x):
    ''' Читает таблицу из РПД в список'''
    ls =[]
    try:
        for row in x.rows:
            for cell in row.cells:
                celltext = ''
                for paragraph in cell.paragraphs:
                    celltext += '\n' + paragraph.text
                ls.append(celltext)
        return [ls, len(x.rows), len(x.columns)]
    except IndexError:
        return [[' '], 1, 1]

cases={'отл':'Отлично', 'хор':'Хорошо', 'неудов':'неУдов', 
       'удовл':'Удовл','не зач':'не Зачтено', 
       'незач':'не Зачтено', 'зач':'Зачтено'}
testtypes = {'the':'Перевод, лексико-грамматический анализ, объем словарного запаса и аннотирование научной статьи',
       'экзамен':'Экзаменационные билеты','зачет':'Задания к зачету'}
       

def normalize(alist):
    thelist = []
    for item in alist:
         item = item.lower()  
         for key, value in cases.items():
              if key in item: item = value
         if not item in cases.values(): 
              item = ' '
         thelist.append(item)
    return thelist

def findout_fos_part(documento):
    ''' Выдергивает все абзацы между литературой и "Примерные контрольные задания" '''
    text_at_start = ['Примерные контрольные задания (вопросы']
    text_at_end = ['учебной литературы', 'iprbook', 'lanbook', 'НБ СВФУ',
                   'информационно-телекоммуникационной сети']
    flag, result_text = False, ''
    for paragraph in documento.paragraphs:
        if any(key_word in paragraph.text for key_word in text_at_end):
          flag = False
        if flag:
          result_text += (paragraph.text+'\n')
        if text_at_start[0] in paragraph.text:
          flag = True
    result_text = result_text.replace('\n\n','\n').replace('  ',' ',1000)    
    return result_text

def find_rpd(subjectcode,subjectname,control,controls,sem):
    global fileslist
    filename = difflib.get_close_matches(subjectcode + ' ' + subjectname,fileslist)
    if len(filename)<1:
        return [' ',' ',' ',' ',' ',' ']
    filename = './rpds/'+filename[0]+'.docx'
    rpd_doc = docx.Document(filename)
    marks = []
    crirs = []
    zuv = ' '   
    zuv_not_found, zuv_not_found2 = True, True
    for table in rpd_doc.tables:
        tablen = table2list(table)
        row, column = tablen[1], tablen[2]
        tablen = tablen[0]
        if len(tablen)<column: 
             continue
        else: 
             row = len(tablen) // column
             tablen = tablen[:(row*column)]
        df = pd.DataFrame(np.array(tablen).reshape(row,column))  #reshape to the table shape


        if sum([key_word in df.iloc[0,column-1] for key_word in ['Шкал','Оценк']])>0:
               marks += list(df[df.columns[-1]])
               crirs += list(df[df.columns[-2]])
        try:
              if zuv_not_found:
                  for irow in range(3,0,-1):
                      for icol in range(1,3):
                            if '(ЗУВ)' in df.iloc[irow,icol]: 
                                zuv += df.iloc[irow+1,icol]
                                zuv_not_found = False

              if zuv_not_found2 and zuv_not_found:                   
                  for irow in range(3,0,-1):
                      for icol in range(1,3):
                             if '.1.2.' in df.iloc[irow,icol]: 
                                zuv += df.iloc[irow+1,icol]
                                zuv_not_found2 = False
        except IndexError: 
              cndekc = 7 

    marks = normalize(marks)
    crirs = [x for ind,x in enumerate(crirs) if marks[ind]!=' ']
    marks = [x for x in marks if x!=' ']
    excellent, good, fair, bad, tests = ' ', ' ', ' ', ' ', ' ' 
    if control=='Зачет':
       for ind, criteria in enumerate(crirs):
           if marks[ind] == 'Зачтено': excellent += crirs[ind]
           if marks[ind] == 'не Зачтено': bad += crirs[ind]
    else:
       for ind, criteria in enumerate(crirs):
           if marks[ind] == 'Отлично': excellent += crirs[ind]
           if marks[ind] == 'Хорошо': good += crirs[ind]
           if marks[ind] == 'Удовл': fair += crirs[ind]
           if marks[ind] == 'не Удов': bad += crirs[ind]
    
    if bad ==' ': bad = excellent
    document = Document(filename)
    bigtext = findout_fos_part(document).lower()
    testtype = ' '
    for key, value in testtypes.items():
              if key in bigtext: 
                   testtype = value
                   continue

    return [zuv.replace('\n\n','\n'),excellent,good,fair,bad,testtype]
 

def fill_table_2(template: DocxTemplate, context: Dict[str, any]) -> None:
    """ Заполнение таблиц с формами контроля """
    global fileslist 
    control_fancy_name = {
        core.CT_EXAM: 'Экзамен',
        core.CT_CREDIT_GRADE: 'Зачет с оценкой',
        core.CT_CREDIT: 'Зачет',
        core.CT_COURSEWORK: 'Курсовой проект',
    }

    plan: core.EducationPlan = context['plan']
    table: Table = template.get_docx().tables[2]
    fileslist = [filename[:-5] for filename in os.listdir('./rpds') if filename.endswith('.docx')]

    row_number = 0
    for competence in sorted(plan.competence_codes.values(), key=core.Competence.repr):
        core.add_table_rows(table, 1)
        row = len(table.rows) - 1
        row_number += 1
        core.set_cell_text(table, row, 0, core.CENTER, str(row_number))
        core.set_cell_text(table, row, 1, core.JUSTIFY, competence.code + ' ' + competence.description)
        for runover in range(2,8):
            core.set_cell_text(table, row, runover, core.JUSTIFY, ' ')
        # table.cell(row, 1).merge(table.cell(row, len(table.columns) - 1))
        subjects = [plan.subject_codes[s] for s in competence.subjects]
        for subject in sorted(subjects, key=core.Subject.repr):
            core.add_table_rows(table, 1)
            row = len(table.rows) - 1
            # len(table.rows) - 1
            # row_number += 1
            controls = []
            for number, semester in subject.semesters.items():
                controls += [control_fancy_name[c] for c in semester.control]
             
            sem = 0
            for control in controls: 
                zuv_criteria = find_rpd(subject.code,subject.name,control,controls,sem)
                sem +=1
                if len(zuv_criteria)==6:
                    core.set_cell_text(table, row, 0, core.CENTER, ' ')
                    core.set_cell_text(table, row, 1, core.JUSTIFY, subject.code + ' ' + subject.name)
                    core.set_cell_text(table, row, 2, core.JUSTIFY, zuv_criteria[0])
                    if control == {'Зачет'}:
                        core.set_cell_text(table, row, 3, core.JUSTIFY,  zuv_criteria[1])
                        core.set_cell_text(table, row, 4, core.JUSTIFY, ' ')
                        core.set_cell_text(table, row, 5, core.JUSTIFY, ' ')
                        core.set_cell_text(table, row, 6, core.JUSTIFY, zuv_criteria[4])
                        table.cell(row, 3).merge(table.cell(row, 5))                
                    else:
                        core.set_cell_text(table, row, 3, core.JUSTIFY, zuv_criteria[1])
                        core.set_cell_text(table, row, 4, core.JUSTIFY, zuv_criteria[2])
                        core.set_cell_text(table, row, 5, core.JUSTIFY, zuv_criteria[3])
                        core.set_cell_text(table, row, 6, core.JUSTIFY, zuv_criteria[4])
                    core.set_cell_text(table, row, 7, core.JUSTIFY, control+' '+zuv_criteria[5])
           
            
    core.fix_table_borders(table)

 
def main() -> None:
    """ Точка входа """
    global fileslist
    check_args()
    plan = core.get_plan('010301_20-1М--.plx') #(sys.argv[1])
    template = core.get_template('fos.docx')
    context = {
        'plan': plan,
    }
    # fill_table_1(template, context)

    fileslist = [filename[:-4] for filename in os.listdir('./rpds') if filename.endswith('.docx')]
     

    fill_table_2(template, context)
    #fill_table_3(template, context)
    #fill_table_4(template, context)
    template.render(context)
    template.save('result.docx') #('_'+sys.argv[2])
    print('Partially done')


if __name__ == '__main__':
    fileslist = ' '
    main()



In [0]:
!python get_fos.py 010301_20-1М--.plx fos.docx
# 'Б1.В.10 Теория выбора и принятия решения ФГОС.docx'[-4:]

Traceback (most recent call last):
  File "get_fos.py", line 552, in <module>
    main()
  File "get_fos.py", line 544, in main
    fill_section_2_2(template, context)
  File "get_fos.py", line 457, in fill_section_2_2
    document.add_paragraph(line)
  File "/usr/local/lib/python3.6/dist-packages/docx/document.py", line 56, in add_paragraph
    return self._body.add_paragraph(text, style)
  File "/usr/local/lib/python3.6/dist-packages/docx/blkcntnr.py", line 37, in add_paragraph
    paragraph.add_run(text)
  File "/usr/local/lib/python3.6/dist-packages/docx/text/paragraph.py", line 37, in add_run
    run.text = text
  File "/usr/local/lib/python3.6/dist-packages/docx/text/run.py", line 163, in text
    self._r.text = text
  File "/usr/local/lib/python3.6/dist-packages/docx/oxml/text/run.py", line 104, in text
    _RunContentAppender.append_to_run_from_text(self, text)
  File "/usr/local/lib/python3.6/dist-packages/docx/oxml/text/run.py", line 134, in append_to_run_from_text
    append

In [0]:
!zip archive *.*

  adding: 010301_20-1М--.plx (deflated 95%)
  adding: 010301_20-1М--.txt (deflated 73%)
  adding: core.py (deflated 73%)
  adding: extractor.py (deflated 66%)
  adding: fos.docx (deflated 5%)
  adding: fozsh.docx (deflated 6%)
  adding: get_fos.py (deflated 75%)
  adding: get_matrix.py (deflated 56%)
  adding: get_rpd.py (deflated 77%)
  adding: README.md (deflated 66%)
  adding: requirements.txt (deflated 4%)
  adding: rpd.docx (deflated 11%)
  adding: rpd_example.yaml (deflated 64%)
  adding: somerpds.zip (stored 0%)
  adding: somerup.zip (stored 0%)


In [0]:
!pip list | grep diff

In [0]:

def fill_table_4(template: DocxTemplate, context: Dict[str, any]) -> None:
    """ Заполнение бланка "Лист сформированности компетенций" """
    plan: core.EducationPlan = context['plan']
    table: Table = template.get_docx().tables[4]
    row_number = 0
    for competence in sorted(plan.competence_codes.values(), key=core.Competence.repr):
        core.add_table_rows(table, 1)
        row_index = len(table.rows) - 1
        row_number += 1
        core.set_cell_text(table, row_index, 0, core.CENTER, str(row_number))
        core.set_cell_text(table, row_index, 1, core.JUSTIFY, competence.code + ' ' + competence.description)
        subjects = [plan.subject_codes[s] for s in competence.subjects]
        for subject in sorted(subjects, key=core.Subject.repr):
            core.add_table_rows(table, 1)
            row_index = len(table.rows) - 1
            core.set_cell_text(table, row_index, 1, core.JUSTIFY, subject.code + ' ' + subject.name)

    core.add_table_rows(table, 1)
    row_number += 1
    row_index = len(table.rows) - 1
    core.set_cell_text(table, row_index, 0, core.CENTER, str(row_number))
    core.set_cell_text(table, row_index, 1, core.JUSTIFY, 'Практики')

    core.add_table_rows(table, 1)
    row_number += 1
    row_index = len(table.rows) - 1
    core.set_cell_text(table, row_index, 0, core.CENTER, str(row_number))
    core.set_cell_text(table, row_index, 1, core.JUSTIFY, 'НИР')

    core.fix_table_borders(table)


NameError: ignored